# Police API Demo

The https://data.police.uk/docs/ website provides an API — an *application programming interface* — that allows us to write programmatic code that can be used to query and download data from the website directly (that is, at a "machine to machine" level).

A Pyhton package exists that "wraps" this API and allows us to call it from Python code: https://github.com/rkhleics/police-api-client-python ([docs](https://police-api-client-python.readthedocs.io/en/latest/)).

## Using the API

Let's see how to use the API to download all the data for a particular year in a particular policie neighbourhood area.

In [1]:
#Initialisation
from police_api import PoliceAPI
api = PoliceAPI()

In [2]:
#Get forces
forces = api.get_forces()
forces

[<Force> Avon and Somerset Constabulary,
 <Force> Bedfordshire Police,
 <Force> Cambridgeshire Constabulary,
 <Force> Cheshire Constabulary,
 <Force> City of London Police,
 <Force> Cleveland Police,
 <Force> Cumbria Constabulary,
 <Force> Derbyshire Constabulary,
 <Force> Devon & Cornwall Police,
 <Force> Dorset Police,
 <Force> Durham Constabulary,
 <Force> Dyfed-Powys Police,
 <Force> Essex Police,
 <Force> Gloucestershire Constabulary,
 <Force> Greater Manchester Police,
 <Force> Gwent Police,
 <Force> Hampshire Constabulary,
 <Force> Hertfordshire Constabulary,
 <Force> Humberside Police,
 <Force> Kent Police,
 <Force> Lancashire Constabulary,
 <Force> Leicestershire Police,
 <Force> Lincolnshire Police,
 <Force> Merseyside Police,
 <Force> Metropolitan Police Service,
 <Force> Norfolk Constabulary,
 <Force> North Wales Police,
 <Force> North Yorkshire Police,
 <Force> Northamptonshire Police,
 <Force> Northumbria Police,
 <Force> Nottinghamshire Police,
 <Force> Police Service of

Each force has several atributes associated with it.

The items that are returned are a little confusing to work with. Let's make them more directly useable:

In [3]:
def getIdName(records, name):
    ''' Get id force by name. '''
    return {r.name:r.id for r in records if name.lower() in r.name.lower()}

In [4]:
getIdName(forces, 'Hampshire')

{'Hampshire Constabulary': 'hampshire'}

In [5]:
#Get police authority
force = api.get_force('hampshire')


In [6]:
def getNameId(records, rid=None):
    ''' Get record name by id. '''
    if rid:
        return {r.name:r.id for r in records if rid == r.id}
    return {r.name:r.id for r in records}

In [7]:
areas = getNameId(force.neighbourhoods)
areas

{'Aldershot North': '14RA01',
 'Aldershot South': '14RA02',
 'Alton': '8OL01',
 'Alton Rural': '8OL02',
 'Andover East': '11TA04',
 'Andover North': '11TA02',
 'Andover South': '11TA03',
 'Andover Town': '11TA01',
 'Bargate': '2SC02',
 'Barncroft and Bedhampton': '7JH02',
 'Basingstoke Centre': '12BB02',
 'Basingstoke East': '12BB05',
 'Basingstoke North': '12BB03',
 'Basingstoke Rural East': '12BL02',
 'Basingstoke Rural South': '12BL01',
 'Basingstoke Rural West': '12BL03',
 'Basingstoke South': '12BB01',
 'Basingstoke West': '12BB04',
 'Battins and West Leigh': '7JH03',
 'Bevois': '2SC01',
 'Bishops Waltham and Soberton': '9WG02',
 'Bitterne North': '2SN02',
 'Bordon': '8OW02',
 'Butser Clanfield Rowlands Castle': '8OP02',
 'Central Southsea and St Jude': '5PS02',
 'Charles Dickens and Nelson': '5PC01',
 'Cosham': '5PN01',
 'Cowes': '6LW01',
 'Coxford Redbridge Millbrook': '2SW02',
 'Crofton and Titchfield': '3FP01',
 'Denmead and Southwick': '9WG05',
 'East Cowes and Wootton': '6LE

In [8]:
#Show dates available
print( api.get_dates() )

['2019-01', '2018-12', '2018-11', '2018-10', '2018-09', '2018-08', '2018-07', '2018-06', '2018-05', '2018-04', '2018-03', '2018-02', '2018-01', '2017-12', '2017-11', '2017-10', '2017-09', '2017-08', '2017-07', '2017-06', '2017-05', '2017-04', '2017-03', '2017-02', '2017-01', '2016-12', '2016-11', '2016-10', '2016-09', '2016-08', '2016-07', '2016-06', '2016-05', '2016-04', '2016-03', '2016-02']


We can get crimes in a particular boundary area defined by a simple boundary definition list (a closed list of points defining an area).

We can get a list for a particular neighbourhood in the following way:

In [9]:
neighbourhood = force.get_neighbourhood( areas['Ryde'] )
boundary = neighbourhood.boundary


We can plot that boundary using folium if we cast it into the correct form.

In particular, we need to convert the list of lat/lon co-ordinates to a set of lon/lat co-ordinates inside a geojson data structure that folium can work with.

(Getting the correct order for co-ordinates can be a pain, with different packages and formats adopting different conventions. A handy summary can be found [here](https://macwright.org/lonlat/).)

In [10]:
def geoJsonFromNeighbourhoodBoundary(boundary):
    ''' Generate geojson polygon from police API boundary. '''
    geojson = {"type": "FeatureCollection",
               "features": [
                   {
                       "type": "Feature",
                       "properties": {},
                       "geometry": {
                           "type": "Polygon",
                           "coordinates":  [[[ll[1],ll[0]] for ll in boundary]]
                       }
                   }
               ]
              }
    return geojson
#geojson

In [13]:
import folium
import json

m = folium.Map(
    location=boundary[0], #Just pick the first point in boundary to centre map - rough and ready!
    #Note that the default location is in lat/lon order
    tiles='Mapbox Bright',
    zoom_start=11
)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary),
    name='geojson'
).add_to(m)

m

## Get crimes in an area

We can use the API to retrieve crimes recorded within a particular area for a particular time period.

In [18]:
crimes = api.get_crimes_area(neighbourhood.boundary, date='2019-01')


In [32]:
#Unpack location details
crimes[0].location.id, crimes[0].location.latitude, crimes[0].location.longitude, crimes[0].location.name


(763767, '50.727222', '-1.162537', 'On or near Supermarket')

In [33]:
#Upack category details
crimes[0].category.id, crimes[0].category.name

('anti-social-behaviour', 'Anti-social behaviour')

In [59]:
import pandas as pd


def getCrimesAsDataFrame(crimes, df=None):
    ''' Convert crimes result to dataframe. '''
    if df is None:
        df=pd.DataFrame(columns = ['cid','type', 'month','name','lid','location','lat','lon'])
        #[int, object, object, int, object, float, float]
    for c in crimes:
        df = df.append({'cid':c.id,'type':c.category.id, 'month':c.month, 'name':c.category.name,
                            'lat':c.location.latitude,'lon':c.location.longitude, 
                        'lid':c.location.id,'location':c.location.name }, ignore_index=True)

    return df

In [60]:
df = getCrimesAsDataFrame(crimes)
df.head()

,cid,type,month,name,lid,location,lat,lon
0,71249494,anti-social-behaviour,2019-01,Anti-social behaviour,763767,On or near Supermarket,50.727222,-1.162537
1,71249516,anti-social-behaviour,2019-01,Anti-social behaviour,763719,On or near West Place,50.726292,-1.167016
2,71249593,anti-social-behaviour,2019-01,Anti-social behaviour,763566,On or near Sutton Close,50.718427,-1.152494
3,71249583,anti-social-behaviour,2019-01,Anti-social behaviour,763545,On or near Slade Road,50.720485,-1.156055
4,71249466,anti-social-behaviour,2019-01,Anti-social behaviour,763785,On or near George Street,50.728725,-1.161490


## Plot Crimes on Map

As in ??, we can plot the crime markers on a map:

Or we can create a heatmap:

Or we can create a heatmap that is animated over several time periods.

To do this, we first need to obtain crime data recorded over several different months: